<a href="https://colab.research.google.com/github/Ashishantal/AI_smile_score/blob/main/ai_score_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle opencv-python tensorflow matplotlib


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("chazzer/smiling-or-not-face-data")

print("Path to dataset files:", path)

100%|██████████| 18.0M/18.0M [00:00<00:00, 78.8MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/chazzer/smiling-or-not-face-data/versions/1


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
DATASET_PATH = "/root/.cache/kagglehub/datasets/chazzer/smiling-or-not-face-data/versions/1"

IMG_SIZE = 64
X = []
y = []

In [ ]:
for label in ["non_smile", "smile"]:
    folder_path = os.path.join(DATASET_PATH, label)
    target = 0 if label == "non_smile" else 1

    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)

        if img is None:
            continue

        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        X.append(img)
        y.append(target)

X = np.array(X) / 255.0
y = np.array(y)

print("Total images:", X.shape)
print("Total labels:", y.shape)


Total images: (1203, 64, 64, 3)
Total labels: (1203,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Training samples:", X_train.shape)
print("Testing samples:", X_test.shape)

Training samples: (962, 64, 64, 3)
Testing samples: (241, 64, 64, 3)


In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,625,281 (6.20 MB)

 Trainable params: 1,625,281 (6.20 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=15,
    batch_size=32
)


Epoch 1/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 7s 83ms/step - accuracy: 0.5351 - loss: 0.7647 - val_accuracy: 0.7220 - val_loss: 0.6017
Epoch 2/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7371 - loss: 0.5471 - val_accuracy: 0.9087 - val_loss: 0.2813
Epoch 3/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9330 - loss: 0.2510 - val_accuracy: 0.9253 - val_loss: 0.2083
Epoch 4/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9424 - loss: 0.1757 - val_accuracy: 0.9295 - val_loss: 0.1952
Epoch 5/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9311 - loss: 0.1756 - val_accuracy: 0.9378 - val_loss: 0.2108
Epoch 6/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9263 - loss: 0.2138 - val_accuracy: 0.8548 - val_loss: 0.3646
Epoch 7/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9460 - loss: 0.1612 - val_accuracy: 0.9295 - val_loss: 0.2706
Epoch 8/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9633 - loss: 0.1103 - val_accuracy: 0.9378 - val_

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9464 - loss: 0.1785 
Test Accuracy: 94.61%


In [ ]:
def smile_score(pred):
    return int(pred * 100)

In [ ]:
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

def detect_face(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    if len(faces) == 0:
        return None

    x, y, w, h = faces[0]
    return img[y:y+h, x:x+w]


In [ ]:
def is_blurry(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    score = cv2.Laplacian(gray, cv2.CV_64F).var()
    return score < 100

In [ ]:
def predict_smile(image_path):
    img = cv2.imread(image_path)

    if img is None:
        return "❌ Image not found"

    face = detect_face(img)
    if face is None:
        return "❌ Cannot detect face. Upload a clear face image."

    if is_blurry(face):
        return "❌ Image is blurry. Please upload a clearer image."

    face = cv2.resize(face, (64,64))
    face = face / 255.0
    face = np.expand_dims(face, axis=0)

    prediction = model.predict(face)[0][0]
    score = smile_score(prediction)

    return f"😊 Smile Score: {score}/100"


In [ ]:
TEST_PATH = "/root/.cache/kagglehub/datasets/chazzer/smiling-or-not-face-data/versions/1/test"


In [ ]:
import os
import cv2
import numpy as np

IMG_SIZE = 64

X_test = []
image_names = []

for img_name in os.listdir(TEST_PATH):
    img_path = os.path.join(TEST_PATH, img_name)

    img = cv2.imread(img_path)
    if img is None:
        continue

    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0

    X_test.append(img)
    image_names.append(img_name)

X_test = np.array(X_test)

print("Total test images:", X_test.shape)


Total test images: (12030, 64, 64, 3)


In [ ]:
predictions = model.predict(X_test, batch_size=32)
smile_scores = (predictions * 100).astype(int)


376/376 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


In [ ]:
for i in range(10):
    print(f"{image_names[i]}  -->  Smile Score: {smile_scores[i][0]}/100")


Marty_Mornhinweg_0001.jpg  -->  Smile Score: 65/100
Lisa_Raymond_0001.jpg  -->  Smile Score: 0/100
Michael_Rolinee_0001.jpg  -->  Smile Score: 0/100
Frank_Solich_0003.jpg  -->  Smile Score: 0/100
Hans_Blix_0001.jpg  -->  Smile Score: 0/100
Deepa_Mehta_0001.jpg  -->  Smile Score: 99/100
Donald_Rumsfeld_0104.jpg  -->  Smile Score: 0/100
Colin_Powell_0233.jpg  -->  Smile Score: 87/100
Turner_Stevenson_0001.jpg  -->  Smile Score: 99/100
Martha_Burk_0001.jpg  -->  Smile Score: 14/100


In [ ]:
import pandas as pd

df = pd.DataFrame({
    "image_name": image_names,
    "smile_score": smile_scores.flatten()
})

df.to_csv("test_smile_predictions.csv", index=False)
print("Saved: test_smile_predictions.csv")


Saved: test_smile_predictions.csv


In [ ]:
from google.colab import files
files.download("test_smile_predictions.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model.save("smile_cnn_model.keras")
print("Model saved as smile_cnn_model.keras")

Model saved as smile_cnn_model.keras


In [ ]:
from google.colab import files
files.download("smile_cnn_model.keras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>